In [1]:
using CSV, DataFrames, NamedArrays, Combinatorics, LinearAlgebra, StatsBase

In [44]:
# import Pkg; Pkg.add("Combinatorics")
# using StatsBase

In [2]:
data = CSV.read("BM93e3.csv", DataFrame)

,blk,A,B,C,D,E,F,G,H,y
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,-1,-1,-1,-1,1,1,1,-1,1,14.0
2,-1,1,-1,-1,-1,-1,1,1,1,16.8
3,-1,-1,1,-1,-1,1,-1,1,1,15.0
4,-1,1,1,-1,1,-1,-1,-1,1,15.4
5,-1,-1,-1,1,1,-1,-1,1,1,27.6
6,-1,1,-1,1,-1,1,-1,-1,1,24.0
7,-1,-1,1,1,-1,-1,1,-1,1,27.4
8,-1,1,1,1,1,1,1,1,1,22.6
9,-1,1,1,1,-1,-1,-1,1,-1,22.3


In [64]:
X = data[1:16, [1, 2, 4, 6, 9]]
y = data[1:16, 10]

16-element Vector{Float64}:
 14.0
 16.8
 15.0
 15.4
 27.6
 24.0
 27.4
 22.6
 22.3
 17.1
 21.5
 17.5
 15.9
 21.9
 16.7
 20.3

In [65]:
p_mod = [0.2356,0.2356,0.2356,0.2356,0.0566]

5-element Vector{Float64}:
 0.2356
 0.2356
 0.2356
 0.2356
 0.0566

In [66]:
fac_mod = NamedArray([2 3 4 0; 1 3 4 0; 1 2 3 0; 1 2 4 0; 1 2 3 4])
setnames!(fac_mod, ["f1", "f2", "f3", "f4"], 2)  
fac_mod

5×4 Named Matrix{Int64}
A ╲ B │ f1  f2  f3  f4
──────┼───────────────
1     │  2   3   4   0
2     │  1   3   4   0
3     │  1   2   3   0
4     │  1   2   4   0
5     │  1   2   3   4

In [67]:
# Quiero arreglar esta construcción, se ve horrible
Xcand = NamedArray([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1] [-1,-1,-1,-1,1,1,1,1,-1,-1,-1,-1,1,1,1,1] [-1,-1,1,1,-1,-1,1,1,-1,-1,1,1,-1,-1,1,1] [-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1] [-1,1,1,-1,1,-1,-1,1,1,-1,-1,1,-1,1,1,-1]])
setnames!(Xcand, ["blk", "f1", "f2", "f3", "f4"], 2)
Xcand

16×5 Named Matrix{Int64}
A ╲ B │ blk   f1   f2   f3   f4
──────┼────────────────────────
1     │   1   -1   -1   -1   -1
2     │   1   -1   -1    1    1
3     │   1   -1    1   -1    1
4     │   1   -1    1    1   -1
5     │   1    1   -1   -1    1
6     │   1    1   -1    1   -1
7     │   1    1    1   -1   -1
8     │   1    1    1    1    1
9     │   1   -1   -1   -1    1
10    │   1   -1   -1    1   -1
11    │   1   -1    1   -1   -1
12    │   1   -1    1    1    1
13    │   1    1   -1   -1   -1
14    │   1    1   -1    1    1
15    │   1    1    1   -1    1
16    │   1    1    1    1   -1

In [68]:
# To drop columns
dropcol(M::NamedArray, j) = M[:, deleteat!(collect(axes(M, 2)), j)]
dropcol(M::DataFrame, j) = M[:, deleteat!(collect(axes(M, 2)), j)]

# Para quitar el primer elemento de un string
snip(s::String) = s[nextind(s,1):end]
snip(s::SubString) = s[nextind(s,1):end]


snip (generic function with 2 methods)

In [78]:
nMod = 5
max_int = 2
g = 2
nStart = 25
nFDes = 4
Iter = 20    # 20

20

In [79]:
# 1
n = size(y, 1)
fac = size(X, 2) - 1
k = size(fac_mod, 2)

# Todavia no estoy segura si los dejare como listas o mejor como vectores
Si = Vector{Float64}(undef, nMod)
Xi = map(_ -> DataFrame(), 1:nMod) # Vector de Dataframes
betai = Vector{Vector{Float64}}(undef,nMod)
gammai = Matrix{Float64}[]  # Vector de matrices
efectos = Vector{Vector{Int8}}(undef,nMod) # Vector de vectores

models = zeros(nMod, fac)
for i = 1:nMod
    aux = fac_mod[i, :]
    filter!(e -> e != 0, aux)
    models[i, aux] .= 1
end

Xfac = dropcol(X, 1)
Xc = dropcol(Xcand, 1)

16×4 Named Matrix{Int64}
A ╲ B │ f1  f2  f3  f4
──────┼───────────────
1     │ -1  -1  -1  -1
2     │ -1  -1   1   1
3     │ -1   1  -1   1
4     │ -1   1   1  -1
5     │  1  -1  -1   1
6     │  1  -1   1  -1
7     │  1   1  -1  -1
8     │  1   1   1   1
9     │ -1  -1  -1   1
10    │ -1  -1   1  -1
11    │ -1   1  -1  -1
12    │ -1   1   1   1
13    │  1  -1  -1  -1
14    │  1  -1   1   1
15    │  1   1  -1   1
16    │  1   1   1  -1

In [80]:
# 2
models = NamedArray(models)
if max_int > 1
    comb = hcat(collect(combinations(1:fac,2))...)
    mat = zeros(Int8, size(models, 1), size(comb, 2))
    for j = 1:size(comb, 2)
        fac1 = comb[1, j]
        fac2 = comb[2, j]
        aux = models[:, fac1] + models[:, fac2] .== 2
        filter!(e -> e != 0, aux)
        cols = names(aux, 1)
        cols = parse.(Int8, cols)
        mat[cols, j] .= 1
        Xfac = hcat(Xfac, Xfac[:, fac1].*Xfac[:, fac2], makeunique = true)    
        Xc = hcat(Xc, Xc[:, fac1].*Xc[:, fac2])
    end
    mat = NamedArray(mat)
    models = hcat(models, mat)
end


5×10 Named Matrix{Float64}
A ╲ hcat │   1    2    3    4    5    6    7    8    9   10
─────────┼─────────────────────────────────────────────────
1        │ 0.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0
2        │ 1.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0
3        │ 1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  0.0
4        │ 1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  0.0
5        │ 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [81]:
# 3
if max_int > 2
    comb = hcat(collect(combinations(1:fac,3))...)
    mat = zeros(Int8, size(models, 1), size(comb, 2))
    for j = 1:size(comb, 2)
        fac1 = comb[1, j]
        fac2 = comb[2, j]
        fac3 = comb[3, j]
        aux = models[:, fac1] + models[:, fac2] + models[:, fac3] .== 3
        filter!(e -> e != 0, aux)
        cols = names(aux, 1)
        cols = parse.(Int8, cols)
        mat[cols, j] .= 1
        Xfac = hcat(Xfac, Xfac[:, fac1].*Xfac[:, fac2].*Xfac[:, fac3], makeunique = true)    
        Xc = hcat(Xc, Xc[:, fac1].*Xc[:, fac2].*Xc[:, fac3])
    end
    mat = NamedArray(mat)
    models = hcat(models, mat)
end

Xfac = hcat(X[:, 1], Xfac, makeunique = true)
Xc = hcat(Xcand[:, 1], Xc)
models = hcat(ones(nMod), models)

5×11 Matrix{Float64}:
 1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0
 1.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [82]:
# 4
models = NamedArray(models)
for i=1:nMod
    aux = models[i, :] .== 1
    filter!(e -> e != 0, aux)
    cols = names(aux, 1)
    cols = parse.(Int8, cols)
    efectos[i] = cols
    tam = length(efectos[i])
    Xi[i] = insertcols!(Xfac[:, efectos[i]], 1,  :efectos => ones(n) , makeunique=true)
    
    mat = zeros(Int8, tam + 1, tam + 1)
    if size(mat, 1) > 1
    # Tengo que hacer este for mas eficiente
        coord = hcat(2:size(mat, 1), 2:size(mat, 1))
        for k=1:(size(mat, 1) - 1)
            mat[coord[k, 1], coord[k, 2]] = 1
        end
    end

    push!(gammai, (1 / g^2)*mat)
    betai[i] = inv(gammai[i] + Matrix(Xi[i])' * Matrix(Xi[i])) * Matrix(Xi[i])' * y
    Si[i] = (y - Matrix(Xi[i]) * betai[i])' * (y - Matrix(Xi[i]) * betai[i]) + betai[i]'*gammai[i] * betai[i]
end
    

In [83]:
# 5
function MDr(extra, X_mat)
    nex = length(extra)
    y_gorro_estrella = Vector{Vector{Float64}}(undef,nMod) #*
    V_estrella = Matrix{Float64}[]
    
    if typeof(X_mat) != Array
        X_mat = Array(X_mat)
    end
    
    for i=1:nMod
        Xiestrella = hcat(ones(nex), X_mat[extra, efectos[i]])
        y_gorro_estrella[i] = Xiestrella * betai[i]
        push!(V_estrella, diagm(0 => ones(nex)) + Xiestrella * inv(gammai[i] + Matrix(Xi[i])'*Matrix(Xi[i])) * Xiestrella')
    end
    
    MD = 0
    m = 1:nMod
    
    for i in m
        for j in m[m .!= i]
            MD = MD + p_mod[i]*p_mod[j]*(-nex + 
                sum(diag(inv(V_estrella[j])*V_estrella[i])) + 
                (n - 1)*(y_gorro_estrella[i] - y_gorro_estrella[j])'*
                inv(V_estrella[j])*(y_gorro_estrella[i] - y_gorro_estrella[j])/Si[i])
        end
    end
    MD = MD * .5
    return(MD)    
end

MDr (generic function with 1 method)

In [29]:
# Pruebas 5

# extra = [16, 1, 12, 1]
# X_mat = Xc
# y_gorro_estrella = Vector{Vector{Float64}}(undef,nMod) #*
# V_estrella = Matrix{Float64}[]

# if (typeof(X_mat) != Array)
#     X_mat = Array(X_mat)
# end
# for i=1:nMod
#     Xiestrella = hcat(ones(nex), X_mat[extra, efectos[i]])
#     y_gorro_estrella[i] = Xiestrella * betai[i]
#     push!(V_estrella, diagm(0 => ones(nex)) + Xiestrella * inv(gammai[i] + Matrix(Xi[i])'*Matrix(Xi[i])) * Xiestrella')
# end
    
# MD = 0
# m = 1:nMod
    
# for i in m
#     for j in m[m .!= i]
#         MD = MD + p_mod[i]*p_mod[j]*(-nex + 
#             sum(diag(inv(V_estrella[j])*V_estrella[i])) + 
#             (n - 1)*(y_gorro_estrella[i] - y_gorro_estrella[j])'*
#             inv(V_estrella[j])*(y_gorro_estrella[i] - y_gorro_estrella[j])/Si[i])
#     end
# end
# MD = MD * .5

In [84]:
# 6
df_MD = DataFrame(DesignPoints = [SubString("0")], MD = Float64(0))
for j in 1:nStart
    extra = sample(1:size(Xcand, 1), nFDes, replace = true)
    iter = 1
    last_out = 0
    last_in = 1
    while last_out != last_in && iter < Iter
        dp = string(sort(extra))
        dp = snip(dp)  # Quita el primer parentesis 
        dp = chop(dp)  # Quita el ultimo parentesis
        
        # No estoy muy segura de esta parte
        aux = df_MD[:, 1] in dp
        if aux != false
            break
        end
        
        # Hasta aquí es mi duda
        df_MD = append!(df_MD, DataFrame(DesignPoints = [dp], MD = round(MDr(extra, Xc); digits = 2)))
        
        op = Vector{Float64}(undef, nFDes)
        for i in 1:nFDes
            op[i] = MDr(extra[1:end .!= i], Xc)  # Aqui hay un error y no se cual es
        end
        index = findmax(op)[2]
        last_out = extra[index]
        extra = extra[1:end .!= index]
        
        op = Vector{Float64}(undef, size(Xcand, 1))
        for i in 1:size(Xcand, 1)
            op[i] = MDr([extra; i],  Xc)
        end
        last_in = findmax(op)[2]
        append!(extra, last_in)
    
        iter = iter + 1
    end
end


In [61]:
# Pruebas 6
df_MD = DataFrame(DesignPoints = [SubString("0")], MD = Float64(0))
# # for j in 1:nStart
extra = sample(1:size(Xcand, 1), nFDes, replace = true)
iter = 1
last_out = 0
last_in = 1
# # #     while last_out != last_in && iter < Iter
dp = string(sort(extra))
dp = snip(dp)  # Quita el primer parentesis 
dp = chop(dp)  # Quita el ultimo parentesis
println("dp:", dp)
#         # No estoy muy segura de esta parte
aux = df_MD[:, 1] in dp
println("aux: ", aux)
# if aux != false
#     break
# end
# #         # Hasta aquí es mi duda
append!(df_MD, DataFrame(DesignPoints = [dp], MD = round(MDr(extra, Xc); digits = 2)))
println(df_MD)

op = Vector{Float64}(undef, nFDes)
for i in 1:nFDes
    op[i] = MDr(extra[1:end .!= i], Xc)  # Aqui hay un error y no se cual es
end
index = findmax(op)[2]
last_out = extra[index]
extra = extra[1:end .!= index]  
println("extra sin index maximo:", extra)
op = Vector{Float64}(undef, size(Xcand, 1))
for i in 1:size(Xcand, 1)
    op[i] = MDr([extra; i],  Xc)
end
last_in = findmax(op)[2]
println("last_in: ", last_in)
append!(extra, last_in)
println("extra sin el segundo maximo", extra)

dp:1, 11, 13, 13
aux: false
2×2 DataFrame
 Row │ DesignPoints   MD      
     │ SubString…     Float64 
─────┼────────────────────────
   1 │ 0                  0.0
   2 │ 1, 11, 13, 13      0.0
extra sin index maximo:[13, 13, 11]
last_in: 1
extra sin el segundo maximo[13, 13, 11, 1]


In [86]:
CSV.write("df_MD.csv", df_MD)

"df_MD.csv"

In [41]:
dp

LoadError: UndefVarError: dp not defined

3×7 Matrix{Int64}:
 -1  -1   1   1  -1  -1   1
 -1  -1  -1   1   1  -1  -1
 -1   1  -1  -1  -1  -1   1